In [1]:
%pip install opencv-python

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Note: you may need to restart the kernel to use updated packages.


In [2]:
import cv2
import numpy as np
import os

def load_your_data():
    sift = cv2.SIFT_create()
    data_path = 'data'
    train_data = []
    test_data = []
    train_labels = []
    test_labels = []
    
    action_map = {
        'boxing': 0,
        'handclapping': 1,
        'handwaving': 2,
        'jogging': 3,
        'running': 4,
        'walking': 5
    }

    for action in os.listdir(data_path):
        if action not in action_map:
            continue
        action_path = os.path.join(data_path, action)
        for person in os.listdir(action_path):
            person_path = os.path.join(action_path, person)
            for video in os.listdir(person_path):
                video_path = os.path.join(person_path, video)
                cap = cv2.VideoCapture(video_path)
                features = []
                ret, prev_frame = cap.read()
                prev_frame = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
                mhi = np.zeros_like(prev_frame, dtype=np.float32)
                
                while True:
                    ret, current_frame = cap.read()
                    
                    if not ret:
                        break
                    
                    current_frame_gray = cv2.cvtColor(current_frame, cv2.COLOR_BGR2GRAY)
                    mhi = motion_recognition.update_mhi(prev_frame, current_frame_gray, mhi)
                    keypoints, descriptors = motion_recognition.extract_sift_features(current_frame)
                    features.append(descriptors)
                    prev_frame = current_frame_gray
                
                cap.release()
                
                if len(features) < 10:
                    # Discard videos that are too short
                    continue
                
                # Split the video features into training and testing sets
                split_index = int(len(features) * 0.8)
                train_data.extend(features[:split_index])
                test_data.extend(features[split_index:])
                train_labels.extend([action_map[action]] * split_index)
                test_labels.extend([action_map[action]] * (len(features) - split_index))
                
    X_train = np.vstack(train_data)
    y_train = np.array(train_labels)
    X_test = np.vstack(test_data)
    y_test = np.array(test_labels)
    
    return X_train, y_train, X_test, y_test

In [3]:
import cv2
import numpy as np
from sklearn import svm
from sklearn.model_selection import KFold, cross_val_score

class MotionRecognition:
    def __init__(self):
        self.sift = cv2.SIFT_create()
        self.mhi_duration = 10

    def extract_sift_features(self, image):
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        keypoints, descriptors = self.sift.detectAndCompute(gray, None)
        return keypoints, descriptors

    def update_mhi(self, prev_frame, current_frame, mhi):
        diff = cv2.absdiff(prev_frame, current_frame)
        _, binary_diff = cv2.threshold(diff, 30, 1, cv2.THRESH_BINARY)

        timestamp = cv2.getTickCount() / cv2.getTickFrequency()
        mhi = cv2.motempl.updateMotionHistory(binary_diff, mhi, timestamp, self.mhi_duration)

        return mhi

    def process_video(self):
        cap = cv2.VideoCapture(0)
        ret, prev_frame = cap.read()

        if not ret:
            raise ValueError("Cannot read the video file.")

        prev_frame = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)

        mhi = np.zeros_like(prev_frame, dtype=np.float32)

        while True:
            ret, current_frame = cap.read()

            if not ret:
                break

            current_frame_gray = cv2.cvtColor(current_frame, cv2.COLOR_BGR2GRAY)
            mhi = self.update_mhi(prev_frame, current_frame_gray, mhi)

            keypoints, descriptors = self.extract_sift_features(current_frame)
            current_frame_with_keypoints = cv2.drawKeypoints(current_frame, keypoints, None)

            cv2.imshow("Current Frame with SIFT keypoints", current_frame_with_keypoints)
            cv2.imshow("MHI", mhi)

            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

            prev_frame = current_frame_gray

        cap.release()
        cv2.destroyAllWindows()

    def train_svm(self, X_train, y_train):
        self.clf.fit(X_train, y_train)

    def predict(self, X_test):
        return self.clf.predict(X_test)

    def k_fold_cross_validation(self, X, y, n_splits=5):
        kf = KFold(n_splits=n_splits)
        scores = cross_val_score(self.clf, X, y, cv=kf)
        return scores.mean(), scores.std()

# Load your training and testing data here
X_train, y_train, X_test, y_test = load_your_data()

motion_recognition = MotionRecognition()

# Train the SVM classifier
motion_recognition.train_svm(X_train, y_train)

# Predict the labels for the test data
y_pred = motion_recognition.predict(X_test)

# Evaluate the classifier using k-fold cross-validation
mean_accuracy, std_accuracy = motion_recognition.k_fold_cross_validation(X_train, y_train)
print(f"Mean accuracy: {mean_accuracy}, Standard deviation: {std_accuracy}")

FileNotFoundError: [WinError 3] The system cannot find the path specified: '/data'